In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import mediapipe as mp 
import cv2 as cv
import os 



In [2]:
## let's load the extracted landmarks 

file_path_Dataset='Top_Classes_Landmarks/Top_Classes_Landmarks'
file_path_Preprocessed='Top_Classes_Landmarks_Preprocessed_No_SlidingWindow_OR_Mask/Top_Classes_Landmarks_Preprocessed_No_SlidingWindow_OR_Mask'


Dataset= []
Dataset_glosses = []


for file in os.listdir(file_path_Dataset):
    if not file.endswith(".npy"):
        continue

    data = np.load(os.path.join(file_path_Dataset, file))
    label = file.split(' ')[0].lower() 

    Dataset.append(data)
    Dataset_glosses.append(label)

Dataset_preprocessed= []
Dataset_preprocessed_glosses = []



for file in os.listdir(file_path_Preprocessed):
    if not file.endswith(".npy"):
        continue

    data = np.load(os.path.join(file_path_Preprocessed, file))
    label = file.split('_')[0].lower()

    Dataset_preprocessed.append(data)
    Dataset_preprocessed_glosses.append(label)

print(len(Dataset_preprocessed), len(Dataset_preprocessed_glosses))





5076 5076


In [3]:
print(len(Dataset), len(Dataset_glosses))

5568 5568


In [4]:
Dataset_preprocessed[600].shape

(157, 438)

In [5]:
Dataset_glosses[7]

'about'

#### Preparing the dataset For feeding to the model 

In [6]:
from sklearn.model_selection import train_test_split

X_train_preprocessed, X_temp, y_train_preprocessed, y_temp = train_test_split(
    Dataset_preprocessed,
    Dataset_preprocessed_glosses,
    test_size=0.10,   
    random_state=42
)

X_val_preprocessed, X_test_preprocessed, y_val_preprocessed, y_test_preprocessed = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,  
    random_state=42
)

In [7]:


X_train_preprocessed = np.array(X_train_preprocessed)
X_val_preprocessed = np.array(X_val_preprocessed)
X_test_preprocessed = np.array(X_test_preprocessed)
y_train_preprocessed = np.array(y_train_preprocessed)
y_val_preprocessed = np.array(y_val_preprocessed)
y_test_preprocessed = np.array(y_test_preprocessed)

In [8]:
X_train_preprocessed.shape

(4568, 157, 438)

In [9]:
len(np.unique(Dataset_glosses))

292

In [10]:
len(np.unique(Dataset_preprocessed_glosses))

132

####  sequence-to-sequence (encoder–decoder) model with attention

In [33]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, GRU, Dense, Embedding,
    Layer, Concatenate
)
from tensorflow.keras.models import Model

# =========================
# PARAMETERS
# =========================
T = X_train_preprocessed.shape[1]   # encoder timesteps
D = X_train_preprocessed.shape[2]   # feature dimension

ENC_UNITS = 1000
VOCAB_SIZE = 132     # number of target words (NOT glosses)
EMB_DIM = 300

# =========================
# ENCODER
# =========================
encoder_inputs = Input(shape=(T, D), name="encoder_inputs")

encoder_outputs, encoder_state = GRU(
    ENC_UNITS,
    return_sequences=True,
    return_state=True,
    dropout=0.2
)(encoder_inputs)

# =========================
# LUONG ATTENTION
# =========================
class LuongAttention(Layer):
    def __init__(self, units):
        super().__init__()
        self.W = Dense(units)

    def call(self, encoder_outputs, decoder_hidden):
        # encoder_outputs: (B, T_enc, U)
        # decoder_hidden:  (B, U)
        
        # Compute score: (B, T_enc, 1)
        score = tf.matmul(
            self.W(encoder_outputs),            # (B, T_enc, U)
            tf.expand_dims(decoder_hidden, -1) # (B, U, 1)
        )
        
        attention_weights = tf.nn.softmax(score, axis=1)  # (B, T_enc, 1)

        # Context vector: weighted sum over encoder outputs
        context = tf.reduce_sum(attention_weights * encoder_outputs, axis=1)  # (B, U)
        return context

# =========================
# CUSTOM LAYER TO REPEAT CONTEXT
# =========================
class RepeatContext(Layer):
    def call(self, inputs):
        context_vector, decoder_outputs = inputs
        # expand context to match decoder timesteps
        context_vector = tf.expand_dims(context_vector, 1)  # (B, 1, U)
        context_vector = tf.tile(
            context_vector,
            [1, tf.shape(decoder_outputs)[1], 1]  # repeat along time dimension
        )
        return context_vector

# =========================
# DECODER
# =========================
decoder_inputs = Input(shape=(None,), name="decoder_inputs")  # variable-length

decoder_emb = Embedding(
    VOCAB_SIZE,
    EMB_DIM,
    mask_zero=True
)(decoder_inputs)

decoder_outputs, _ = GRU(
    ENC_UNITS,
    return_sequences=True,
    return_state=True,
    dropout=0.2
)(decoder_emb, initial_state=encoder_state)

# =========================
# ATTENTION + CONCAT
# =========================
attention = LuongAttention(ENC_UNITS)
context_vector = attention(encoder_outputs, encoder_state)

# dynamically repeat context for each decoder timestep using custom layer
context_vector = RepeatContext()([context_vector, decoder_outputs])

decoder_concat = Concatenate(axis=-1)([decoder_outputs, context_vector])

# =========================
# OUTPUT
# =========================
decoder_outputs = Dense(
    VOCAB_SIZE,
    activation="softmax"
)(decoder_concat)

# =========================
# MODEL
# =========================
model = Model(
    inputs=[encoder_inputs, decoder_inputs],
    outputs=decoder_outputs
)





c:\Users\shahd\miniconda3\envs\sign_env\lib\site-packages\keras\src\layers\layer.py:970: UserWarning: Layer 'repeat_context' (of type RepeatContext) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [34]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_inputs      │ (None, 157, 438)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, None, 300) │     39,600 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_9 (GRU)         │ [(None, 157,      │  4,320,000 │ encoder_inputs[0… │
│                     │ 1000), (None,     │            │                   │
│                     │ 1000)]            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_10 (GRU)        │ [(None, None,     │  3,906,000 │ embedding_6[0][0… │
│                     │ 1000), (None,     │            │ gru_9[0][1]       │
│                     │ 1000)]            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ luong_attention_5   │ (None, 1000)      │  1,001,000 │ gru_9[0][0],      │
│ (LuongAttention)    │                   │            │ gru_9[0][1]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_context      │ (None, None,      │          0 │ luong_attention_… │
│ (RepeatContext)     │ 1000)             │            │ gru_10[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, None,      │          0 │ gru_10[0][0],     │
│ (Concatenate)       │ 2000)             │            │ repeat_context[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, None, 132) │    264,132 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,530,732 (36.36 MB)

 Trainable params: 9,530,732 (36.36 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
import numpy as np

# =========================
# MAKE SURE y_* ARE 2D
# =========================
def ensure_2d(y):
    if len(y.shape) == 1:   # shape (num_samples,)
        y = np.expand_dims(y, 1)
    elif len(y.shape) > 2:
        y = np.squeeze(y)
        if len(y.shape) == 1:
            y = np.expand_dims(y, 1)
    return y

y_train_encoded = ensure_2d(y_train_encoded)
y_val_encoded   = ensure_2d(y_val_encoded)
y_test_encoded  = ensure_2d(y_test_encoded)

# =========================
# DECODER INPUTS & OUTPUTS
# =========================
def prepare_decoder_sequences(y):
    """
    Given target sequences y (integer-encoded), prepare:
    - decoder input: y[:, :-1]  (shifted right)
    - decoder output: y[:, 1:]  (what model predicts)
    """
    # If sequence length is 1, just repeat to avoid empty slices
    if y.shape[1] < 2:
        decoder_input = y
        decoder_output = y
    else:
        decoder_input = y[:, :-1]
        decoder_output = y[:, 1:]
    return decoder_input, decoder_output

# =========================
# PREPARE DECODER SEQUENCES
# =========================
y_train_input, y_train_output = prepare_decoder_sequences(y_train_encoded)
y_val_input, y_val_output     = prepare_decoder_sequences(y_val_encoded)
y_test_input, y_test_output   = prepare_decoder_sequences(y_test_encoded)

# =========================
# PRINT SHAPES TO VERIFY
# =========================
print("X_train shape:", X_train_preprocessed.shape)
print("y_train_input shape:", y_train_input.shape)
print("y_train_output shape:", y_train_output.shape)
print("X_val shape:", X_val_preprocessed.shape)
print("y_val_input shape:", y_val_input.shape)
print("y_val_output shape:", y_val_output.shape)
print("X_test shape:", X_test_preprocessed.shape)
print("y_test_input shape:", y_test_input.shape)
print("y_test_output shape:", y_test_output.shape)


X_train shape: (4568, 157, 438)
y_train_input shape: (4568, 1)
y_train_output shape: (4568, 1)
X_val shape: (254, 157, 438)
y_val_input shape: (254, 1)
y_val_output shape: (254, 1)
X_test shape: (254, 157, 438)
y_test_input shape: (254, 1)
y_test_output shape: (254, 1)


In [43]:
history = model.fit(
    [X_train_preprocessed, y_train_input],
    y_train_output,
    validation_data=([X_val_preprocessed, y_val_input], y_val_output),
    batch_size=32,
    epochs=50
)



Epoch 1/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.0313 - loss: 4.7969 - val_accuracy: 0.0630 - val_loss: 4.6536
Epoch 2/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 275s 2s/step - accuracy: 0.0849 - loss: 4.5627 - val_accuracy: 0.2047 - val_loss: 4.3012
Epoch 3/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.3255 - loss: 3.9437 - val_accuracy: 0.4370 - val_loss: 3.4303
Epoch 4/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 142s 989ms/step - accuracy: 0.6025 - loss: 3.0770 - val_accuracy: 0.9252 - val_loss: 2.5092
Epoch 5/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 138s 966ms/step - accuracy: 0.9656 - loss: 2.1861 - val_accuracy: 0.9961 - val_loss: 1.7067
Epoch 6/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 143s 1000ms/step - accuracy: 0.9967 - loss: 1.3027 - val_accuracy: 0.9961 - val_loss: 0.8770
Epoch 7/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 139s 975ms/step - accuracy: 0.9993 - loss: 0.6046 - val_accuracy: 1.0000 - val_loss: 0.3966
Epoch 8/50
143/143 ━━━━━━━━━━━━━━━━━━━━ 140s 980ms/step - accuracy: 0.9993 - loss: 0

In [44]:
test_loss, test_acc = model.evaluate([X_test_preprocessed, y_test_input], y_test_output)
print("Test accuracy:", test_acc)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 238ms/step - accuracy: 1.0000 - loss: 3.3405e-04
Test accuracy: 1.0
